# Process recount2 data
This notebook does the following:

1. Selects template experiment
2. Downloads subset of recount2 data, including the template experiment (50 random experiments + 1 template experiment)
3. Train VAE on subset of recount2 data

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import random
import rpy2
import seaborn as sns
from sklearn import preprocessing
import pickle

sys.path.append("../")
from ponyo import generate_template_data, utils, pipeline
from functions import process, calc

from numpy.random import seed
random_state = 123
seed(random_state)

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


In [2]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(),"../"))

config_file = os.path.abspath(os.path.join(base_dir,
                                           "config_human.tsv"))
params = utils.read_config(config_file)

### Select template experiment

We manually selected bioproject [SRP012656](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE37764), which contains primary non-small cell lung adenocarcinoma tumors and adjacent normal tissues of 6 never-smoker Korean female patients with 2 replicates each.

In [3]:
# Load params
local_dir = params["local_dir"]
dataset_name = params['dataset_name']
NN_architecture = params['NN_architecture']
project_id = params['project_id']

### Download subset of recount2 to use as a compendium
The compendium will be composed of random experiments + the selected template experiment

In [4]:
%%R
# Select 59
# Select a
# Run one time
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install("derfinder")
#BiocManager::install("recount")

NULL


In [5]:
%%R
library('recount')

In [6]:
#%%R -i project_id -i local_dir

#source('../functions/download_recount2_data.R')

#get_recount2_compendium(project_id, local_dir)

### Download expression data for selected project id

In [7]:
%%R -i project_id -i local_dir

source('../functions/download_recount2_data.R')

get_recount2_template_experiment(project_id, local_dir)

Loading objects:
  rse_gene


### Subset genes
For our downstream analysis we will be comparing our set of differentially expression genes against the set found in [Crow et. al. publication](https://www.pnas.org/content/pnas/116/13/6491.full.pdf), therefore we will limit our genes to include only those genes shared between our starting set of genes and those in publication. 

In [8]:
# Get generic genes identified by Crow et. al.
DE_prior_file = "https://raw.githubusercontent.com/maggiecrow/DEprior/master/DE_Prior.txt"

DE_prior = pd.read_csv(DE_prior_file,
                       header=0,
                       sep="\t")

DE_prior.head()

,Gene_Order,Gene_EntrezID,N_HitLists,DE_Prior_Rank,Gene_Name
0,1,7503,79,1.000000,XIST
1,2,8653,64,0.999948,DDX3Y
2,3,9086,62,0.999870,EIF1AY
3,4,8284,52,0.999870,KDM5D
4,5,8287,46,0.999791,USP9Y


In [9]:
# Get list of published generic genes
published_generic_genes = list(DE_prior['Gene_Name'])

In [10]:
# Get list of our genes
# Load real template experiment
template_data_file = params['template_data_file']

# Read template data
template_data = pd.read_csv(
    template_data_file,
    header=0,
    sep='\t',
    index_col=0)

our_gene_ids = list(template_data.columns)

In [11]:
# File mapping ensembl ids to hgnc symbols
gene_id_file = os.path.join(
    local_dir,
    "ensembl_hgnc_mapping.tsv")

In [12]:
%%R
suppressWarnings(library("biomaRt"))

In [13]:
%%R -i template_data_file -i gene_id_file

# Get mapping between ensembl gene ids (ours) to HGNC gene symbols (published)

source('../functions/process_names.R')

if (file.exists(gene_id_file) == FALSE){
    gene_id_mapping <- get_ensembl_symbol_mapping(template_data_file, gene_id_file)
}

In [14]:
# Read gene id mapping
gene_id_mapping = pd.read_csv(
        gene_id_file,
        header=0,
        sep='\t',
        index_col=0)

print(gene_id_mapping.shape)
gene_id_mapping.head()

(57210, 2)


,ensembl_gene_id,hgnc_symbol
1,ENSG00000002330,BAD
2,ENSG00000003137,CYP26B1
3,ENSG00000003249,DBNDD1
4,ENSG00000004799,PDK4
5,ENSG00000006062,MAP3K14


In [15]:
# Get mapping between ensembl ids with and without version numbers
# Expressiond data uses ensembl gene ids with version number 
ensembl_gene_ids = pd.DataFrame(data={'ensembl_version': our_gene_ids,
                                      'ensembl_parsed': [gene_id.split('.')[0] for gene_id in our_gene_ids]})

print(ensembl_gene_ids.shape)
ensembl_gene_ids.head()

(58037, 2)


,ensembl_version,ensembl_parsed
0,ENSG00000000003.14,ENSG00000000003
1,ENSG00000000005.5,ENSG00000000005
2,ENSG00000000419.12,ENSG00000000419
3,ENSG00000000457.13,ENSG00000000457
4,ENSG00000000460.16,ENSG00000000460


In [16]:
# Map ensembl ids with version number to gene_id_mapping_df
gene_id_mapping = pd.merge(gene_id_mapping, 
                           ensembl_gene_ids, 
                           left_on='ensembl_gene_id',
                           right_on='ensembl_parsed', 
                           how='outer')

print(gene_id_mapping.shape)
gene_id_mapping.set_index('ensembl_version', inplace=True)
gene_id_mapping.head()

(58129, 4)


,ensembl_gene_id,hgnc_symbol,ensembl_parsed
ensembl_version,,,
ENSG00000002330.13,ENSG00000002330,BAD,ENSG00000002330
ENSG00000003137.8,ENSG00000003137,CYP26B1,ENSG00000003137
ENSG00000003249.13,ENSG00000003249,DBNDD1,ENSG00000003249
ENSG00000004799.7,ENSG00000004799,PDK4,ENSG00000004799
ENSG00000006062.13,ENSG00000006062,MAP3K14,ENSG00000006062


Since this experiment contains both RNA-seq and smRNA-seq samples which are in different ranges so we will drop smRNA samples so that samples are within the same range. The analysis identifying these two subsets of samples can be found in this [notebook](0_explore_input_data.ipynb)

In [17]:
# Replace ensembl ids with gene symbols
template_data = process.replace_ensembl_ids(template_data,
                                            gene_id_mapping)

In [18]:
template_data.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,MIR6830,MIR548AH,MYOCOS,MIR522,MIR6715B,MIR3116-2,MIR3202-2,HSFX3,VINAC1P,MIR4481
SRR493937,3.800154,0.018190,9.128459,1.585803,0.713952,84.990542,19.943002,12.065586,2.498822,7.713002,...,0.000000,0.0,0.135649,0.0,0.0,0.00000,0.000000,0.588513,0.0,0.000000
SRR493938,3.798473,0.000000,9.157669,1.456017,0.738967,85.788323,19.621252,11.738855,2.435308,7.125808,...,0.005417,0.0,0.127873,0.0,0.0,0.00000,0.000000,0.452511,0.0,0.000000
SRR493939,20.222134,0.000000,15.522072,3.944919,2.512265,8.072948,47.246078,18.866957,2.443230,12.143670,...,0.000000,0.0,0.011112,0.0,0.0,0.00000,0.000000,0.446833,0.0,0.000000
SRR493940,19.988329,0.000000,17.140304,3.977217,2.378849,7.599225,46.594458,18.921190,2.448276,12.048960,...,0.147412,0.0,0.037886,0.0,0.0,0.07756,0.000000,0.462295,0.0,0.168159
SRR493941,3.239231,0.019398,9.069316,2.181582,1.008155,17.675885,19.371137,11.759756,1.970725,6.740705,...,0.002860,0.0,0.156220,0.0,0.0,0.00000,0.070955,0.739403,0.0,0.000000


In [19]:
# Get intersection of gene lists
our_gene_ids_hgnc = template_data.columns
shared_genes_hgnc = list(set(our_gene_ids_hgnc).intersection(published_generic_genes))
print(len(shared_genes_hgnc))

17785


In [20]:
# Save shared genes
shared_genes_file = os.path.join(
    local_dir,
    "shared_gene_ids.pickle")

outfile = open(shared_genes_file,'wb')
pickle.dump(shared_genes_hgnc,outfile)
outfile.close()

In [21]:
# Drop smRNA samples so that samples are within the same range
smRNA_samples = ["SRR493961",
                 "SRR493962",
                 "SRR493963",
                 "SRR493964",
                 "SRR493965",
                 "SRR493966",
                 "SRR493967",
                 "SRR493968",
                 "SRR493969",
                 "SRR493970",
                 "SRR493971",
                 "SRR493972"]

In [22]:
# Drop samples
template_data = template_data.drop(smRNA_samples)

In [23]:
# Drop genes
template_data = template_data[shared_genes_hgnc]

print(template_data.shape)
template_data.head()

(24, 17788)


,ARHGAP45,ZNF610,PPP1R3B,RBBP7,CCDC185,GPATCH3,C1orf159,DECR2,NAA11,KRTAP4-6,...,FBXL6,UBC,CD1A,OLFML2B,CACNA2D4,EID1,CHRNA10,SCYL2,SCRT1,POLR2L
SRR493937,35.762946,0.271364,7.859956,7.288771,0.000000,3.961903,2.380899,2.577827,0.000000,0.013790,...,7.551714,772.176176,1.139391,7.299552,0.571510,30.028322,1.354713,5.571204,0.010249,75.147459
SRR493938,35.645899,0.201617,7.365964,7.483894,0.012276,3.871496,2.049562,2.116853,0.010950,0.013837,...,7.729311,766.487971,1.244618,7.350872,0.576157,31.325761,1.232702,5.568605,0.024432,73.244201
SRR493939,12.084931,1.146171,7.689973,16.502015,0.115370,4.096778,4.413026,8.868962,0.010627,0.006312,...,18.543304,517.161381,4.463290,5.384466,0.387584,39.226702,0.495944,13.582267,0.065016,100.520026
SRR493940,12.239497,1.104245,7.734640,16.218601,0.139438,4.838954,4.288688,8.957386,0.000000,0.000000,...,17.779922,518.814696,4.515256,6.079031,0.448529,39.771272,0.668580,13.771750,0.068446,94.962601
SRR493941,19.122830,0.722249,6.414456,7.214821,0.000000,3.727227,2.236915,3.578029,0.000000,0.000000,...,7.752204,356.842882,0.599057,6.977339,0.539968,35.109401,0.518152,5.237986,0.005351,104.470958


In [24]:
print(len(template_data.columns) - len(shared_genes_hgnc))

3


*Note:* There is a difference in the number of `shared_genes_hgnc` and genes in the template experiment because 3 genes have 2 different ensembl gene ids have map to the same hgnc symbol (one forward, one reverse)

In [25]:
# Save 
template_data.to_csv(template_data_file, float_format='%.5f', sep='\t')

### Normalize compendium 

In [26]:
# Load real gene expression data
original_compendium_file = params['compendium_data_file']

In [27]:
# Read data
original_compendium = pd.read_table(
    original_compendium_file,
    header=0,
    sep='\t',
    index_col=0)

print(original_compendium.shape)
original_compendium.head()

(1552, 17785)


,GIN1,ATP7B,STX2,CALML4,IL33,NDUFA2,NIPA2,CC2D1B,KRT36,SLC11A2,...,FHL1,FAM217B,SLC35A3,FOXN3-AS2,ADA,MAP2K3,NALT1,TAGLN2,BNIPL,LINC00347
SRR1604987,2.390,0.718,1.672,1.643,0.007,24.263,10.499,9.866,0.001,10.171,...,0.774,4.924,2.727,0.000,6.939,32.849,0.155,237.958,0.070,0.000
SRR1604988,2.462,0.711,1.455,1.586,0.037,24.275,11.261,9.867,0.024,10.181,...,0.702,4.955,2.866,0.013,6.641,33.477,0.209,238.721,0.081,0.000
SRR1604989,1.603,0.814,2.398,2.534,0.010,14.434,15.402,7.834,0.001,8.509,...,0.689,4.520,4.233,0.000,7.546,28.347,0.123,149.598,0.144,0.000
SRR1604990,1.872,0.748,2.193,2.294,0.020,14.673,14.667,7.892,0.003,8.536,...,0.802,4.469,4.414,0.000,7.558,27.931,0.193,152.254,0.183,0.000
SRR1604991,0.786,1.362,6.530,1.390,0.012,18.238,17.125,10.627,0.001,3.683,...,7.346,5.528,2.535,0.000,9.323,32.487,0.311,241.787,0.105,0.004


In [28]:
# Replace ensembl ids with gene symbols
#original_compendium = process.replace_ensembl_ids(original_compendium,
#                                                gene_id_mapping)

In [29]:
# Drop genes
original_compendium = original_compendium[shared_genes_hgnc]

original_compendium.head()

,ARHGAP45,ZNF610,PPP1R3B,RBBP7,CCDC185,GPATCH3,C1orf159,DECR2,NAA11,KRTAP4-6,...,FBXL6,UBC,CD1A,OLFML2B,CACNA2D4,EID1,CHRNA10,SCYL2,SCRT1,POLR2L
SRR1604987,4.686,2.914,6.349,12.617,0.034,4.167,1.487,4.824,0.000,0.107,...,6.677,418.218,0.0,0.070,0.050,40.078,0.194,9.640,0.012,52.679
SRR1604988,4.968,2.993,6.243,12.530,0.011,4.031,1.397,4.778,0.006,0.065,...,6.247,413.291,0.0,0.040,0.042,40.601,0.090,9.914,0.007,51.503
SRR1604989,5.910,2.855,13.585,16.710,0.000,3.882,1.172,2.845,0.000,0.000,...,5.733,297.397,0.0,0.019,0.003,38.530,0.154,10.461,0.002,39.457
SRR1604990,5.737,2.884,14.170,16.927,0.016,3.883,1.199,2.745,0.000,0.000,...,5.552,298.423,0.0,0.038,0.012,37.747,0.114,10.902,0.000,38.281
SRR1604991,2.580,2.653,8.768,17.953,0.067,4.608,1.624,3.556,0.000,0.000,...,5.893,281.337,0.0,0.986,0.012,64.853,0.215,8.097,0.010,83.953


In [30]:
# 0-1 normalize per gene
scaler = preprocessing.MinMaxScaler()
original_data_scaled = scaler.fit_transform(original_compendium)
original_data_scaled_df = pd.DataFrame(original_data_scaled,
                                columns=original_compendium.columns,
                                index=original_compendium.index)

print(original_data_scaled_df.shape)
original_data_scaled_df.head()

(1552, 17785)


,ARHGAP45,ZNF610,PPP1R3B,RBBP7,CCDC185,GPATCH3,C1orf159,DECR2,NAA11,KRTAP4-6,...,FBXL6,UBC,CD1A,OLFML2B,CACNA2D4,EID1,CHRNA10,SCYL2,SCRT1,POLR2L
SRR1604987,0.008520,0.285798,0.049474,0.123779,0.001503,0.111375,0.045315,0.141824,0.000000,0.001282,...,0.089699,0.198485,0.0,0.000616,0.002863,0.069023,0.011408,0.372906,0.000190,0.017902
SRR1604988,0.009032,0.293546,0.048648,0.122925,0.000486,0.107740,0.042572,0.140472,0.000105,0.000779,...,0.083922,0.196147,0.0,0.000352,0.002405,0.069924,0.005292,0.383505,0.000111,0.017503
SRR1604989,0.010745,0.280012,0.105860,0.163933,0.000000,0.103758,0.035715,0.083642,0.000000,0.000000,...,0.077017,0.141144,0.0,0.000167,0.000172,0.066357,0.009056,0.404665,0.000032,0.013409
SRR1604990,0.010430,0.282856,0.110418,0.166062,0.000708,0.103785,0.036538,0.080702,0.000000,0.000000,...,0.074586,0.141631,0.0,0.000334,0.000687,0.065009,0.006704,0.421724,0.000000,0.013009
SRR1604991,0.004691,0.260200,0.068324,0.176127,0.002963,0.123162,0.049490,0.104545,0.000000,0.000000,...,0.079167,0.133522,0.0,0.008671,0.000687,0.111692,0.012643,0.313218,0.000158,0.028531


In [31]:
# Save data
normalized_data_file = params['normalized_compendium_data_file']

original_data_scaled_df.to_csv(
    normalized_data_file, float_format='%.3f', sep='\t')

original_compendium.to_csv(
    original_compendium_file, float_format='%.3f', sep='\t')

# Save scaler transform
scaler_file = params['scaler_transform_file']

outfile = open(scaler_file,'wb')
pickle.dump(scaler,outfile)
outfile.close()

### Train VAE 

In [32]:
# Setup directories
# Create VAE directories
output_dirs = [os.path.join(base_dir, dataset_name, "models"),
               os.path.join(base_dir, dataset_name, "logs")]

# Check if analysis output directory exist otherwise create
for each_dir in output_dirs:
    if os.path.exists(each_dir) == False:
        print('creating new directory: {}'.format(each_dir))
        os.makedirs(each_dir, exist_ok=True)

# Check if NN architecture directory exist otherwise create
for each_dir in output_dirs:
    new_dir = os.path.join(each_dir, NN_architecture)
    if os.path.exists(new_dir) == False:
        print('creating new directory: {}'.format(new_dir))
        os.makedirs(new_dir, exist_ok=True)

In [33]:
# Train VAE on new compendium data
#pipeline.train_vae(config_file,
#                   normalized_data_file)